# AutoEncoder for feature Extraction

In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import torch, torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
from PIL import Image
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from sklearn import decomposition
from sklearn.model_selection import train_test_split

In [ ]:
## checking current directory
directory = os.getcwd()
print(directory)

In [ ]:
## enviroinment setting
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

## Dataset Settings

In [ ]:
# Data Transformation for fNIRS
data_transforms_1 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])
# Data Transformation
data_transforms_2 = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

* Input dataset shapes:
> fNIRS: 682x539 \
> VFT: 720x720

In [ ]:
# Uploading the food image data

data_train = datasets.ImageFolder(root = 'E:/RESEARCH/BRAIN/research_data/FNIRS/VFT_3CLASS/train', transform = data_transforms_1) ## fNIRS
# data_train = datasets.ImageFolder(root = 'E:/RESEARCH/BRAIN/research_data/VFT_3CLASS', transform = data_transforms_2) ##VFT

In [ ]:
## arguments setting for hyperparameter tuning
class Args:
    # arugments
    epochs=150
    bs=32
    lr=0.0001
    momentum=0.9
    
    num_channels=3
    num_classes=2
    verbose='store_true'
    seed=712002

args = Args()    

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)

In [ ]:
## divide the overall dataset into train and test dataset
train_size = int(0.8 * len(data_train))
test_size = len(data_train)-train_size
print('Training dataset size is:', train_size, '/ Test dataset size is:', test_size)

In [ ]:
## train test split for model training
train_dataset, test_dataset = torch.utils.data.random_split(data_train, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.bs, shuffle=False, num_workers=4)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)

In [ ]:
images[0].size()
# images[0]

In [ ]:
# image_reshape = images.reshape([-1, 512*512])
# image_reshape.size()

* Image check

In [ ]:
# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [ ]:
images_np = images.numpy() # convert images to numpy for display

In [ ]:
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(5):
    ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
    imshow(images_np[idx])

## Autoencoder Model with PyTorch

In [ ]:
# ## Convolutional Autoencoder for fNIRS brainimage dataset
# class convAutoencoder_1(nn.Module):
#     def __init__(self):
#         super(convAutoencoder_1, self).__init__()
        
#         # Encoder
#         self.cnn_encoder = nn.Sequential(
#             nn.Linear(512*512, 2048),
#             nn.ReLU(),
#             nn.ReLU(2048, 128)
#         )

#         # Decoder
#         self.cnn_decoder = nn.Sequential(
#             nn.Linear(128, 2048),
#             nn.ReLU(),
#             nn.ReLU(2048, 512*512)
#             nn.sigmoid
#         )
            
            
#     def forward(self, x):
#         encoder_out = self.cnn_encoder(x)
#         decoder_out = self.cnn_decoder(encoder_out)
#         return out
    
#     def get_hidden(self,x):
#         return self.cnn_encoder(x)

In [ ]:
## Convolutional Autoencoder for VFT Vocal recording dataset
class convAutoencoder_1(nn.Module):
    def __init__(self):
        super(convAutoencoder_1, self).__init__()
        
        # Encoder
        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        # Decoder
        self.cnn_decoder = nn.Sequential(
            nn.ConvTranspose2d(2, 16, kernel_size = 2, stride = 2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size = 2, stride = 2, padding=0),
            nn.Sigmoid()
        )
            
            
    def forward(self, x):
        encoder_out = self.cnn_encoder(x)
        decoder_out = self.cnn_decoder(encoder_out)
        return decoder_out
    
    def get_hidden(self,x):
        return self.cnn_encoder(x)

In [ ]:
## Convolutional Autoencoder for VFT Vocal recording dataset
class convAutoencoder_2(nn.Module):
    def __init__(self):
        super(convAutoencoder_2, self).__init__()
        
        # Encoder
        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        # Decoder
        self.cnn_decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 16, kernel_size = 2, stride = 2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size = 2, stride = 2, padding=0),
            nn.Sigmoid()
        )
            
            
    def forward(self, x):
        encoder_out = self.cnn_encoder(x)
        decoder_out = self.cnn_decoder(encoder_out)
        return decoder_out
    
    def get_hidden(self,x):
        return self.cnn_encoder(x)

In [ ]:
# Setting Optimizer and Objective Function
# model = convAutoencoder_1().to(DEVICE)
model = convAutoencoder_1().to(DEVICE)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = args.lr)
print(model)

In [ ]:
for epoch in range(1, args.epochs+1):
    train_loss = 0.0

    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        # calculate the loss
        loss = criterion(output, image)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*image.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

In [ ]:
model.eval()

with torch.no_grad():
    hidden_features = []
    for image, label in test_loader:
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        hidden = model.cnn_encoder(image)
        hidden_features.append(hidden)
    hidden_features = torch.cat(hidden_features, dim=0)

In [ ]:
print("Extracted hidden feature shape is {} by {} vector with {} dimensionality".format(hidden_features.shape[2],hidden_features.shape[3], hidden_features.shape[1]))

In [ ]:
hidden_features[1].shape